In [21]:
import subprocess
import json
import praw
from newsapi import NewsApiClient
from textblob import TextBlob
from datetime import datetime, timedelta

# ----------- API KEYS AND CONFIG -----------
REDDIT_CLIENT_ID = "V8W1O2VI7wiZ1tv96LnzTg"
REDDIT_SECRET = "vlp_w9v35P-29qm-MxpyMxFB3D_mrA"
REDDIT_USER_AGENT = "PiggyLime's Market Bot"

#NEWS_API_KEY = 'your_newsapi_key'

# ----------- INIT APIS -----------
reddit = praw.Reddit(
    client_id="V8W1O2VI7wiZ1tv96LnzTg",
    client_secret="vlp_w9v35P-29qm-MxpyMxFB3D_mrA",
    user_agent="PiggyLime's Market Bot"
)

#newsapi = NewsApiClient(api_key=NEWS_API_KEY)

# ----------- CORE FUNCTIONS -----------

def analyze_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity  # float between -1 and 1

def fetch_tweets_cli(keyword, limit=20):
    print(f"\n🔍 Tweets on: {keyword}")
    tweets = []
    try:
        command = f"snscrape --max-results {limit} --jsonl twitter-search '{keyword} lang:en'"
        result = subprocess.run(command, shell=True, capture_output=True, text=True)
        for line in result.stdout.strip().split('\n'):
            tweet = json.loads(line)
            content = tweet.get("content", "")
            sentiment = analyze_sentiment(content)
            tweets.append((content, sentiment))
    except Exception as e:
        print(f"Error fetching tweets: {e}")
    return tweets

def fetch_reddit(keyword, limit=10):
    print(f"\n🔍 Reddit posts on: {keyword}")
    posts = []
    try:
        for submission in reddit.subreddit("all").search(keyword, limit=limit):
            content = submission.title + " " + submission.selftext
            sentiment = analyze_sentiment(content)
            posts.append((submission.title, sentiment))
    except Exception as e:
        print(f"Error fetching Reddit posts: {e}")
    return posts

def fetch_news(keyword, days=3):
    print(f"\n🔍 News articles on: {keyword}")
    news_data = []
    try:
        from_date = (datetime.now() - timedelta(days=days)).strftime('%Y-%m-%d')
        articles = newsapi.get_everything(q=keyword, from_param=from_date, language='en', sort_by='relevancy', page_size=10)
        for article in articles['articles']:
            content = article['title'] + ' ' + (article['description'] or '')
            sentiment = analyze_sentiment(content)
            news_data.append((article['title'], sentiment))
    except Exception as e:
        print(f"Error fetching news: {e}")
    return news_data

def summarize_insights(data, source):
    total_sentiment = sum(s for _, s in data)
    avg_sentiment = total_sentiment / len(data) if data else 0
    print(f"\n📊 {source} Sentiment Summary:")
    print(f"- Average sentiment: {avg_sentiment:.2f}")
    print("- Top Insights:")
    for text, sentiment in data[:10]:
        print(f"  - ({sentiment:.2f}) {text[:100]}...")

# ----------- MAIN WORKFLOW -----------

def get_market_insights(query):
    #tweets = fetch_tweets_cli(query)
    reddit_posts = fetch_reddit(query)
    #news_articles = fetch_news(query)

    #summarize_insights(tweets, "Twitter")
    summarize_insights(reddit_posts, "Reddit")
    #summarize_insights(news_articles, "News")

# ----------- Run Example -----------

if __name__ == '__main__':
    user_query = input("Enter market topic to research: ")
    get_market_insights(user_query)


Enter market topic to research:  cred rewards



🔍 Reddit posts on: cred rewards

📊 Reddit Sentiment Summary:
- Average sentiment: 0.03
- Top Insights:
  - (0.10) Album of the Year: Kendrick Lamar - GNX...
  - (-0.50) Random person at the bank this morning....
  - (-0.05) DE, pls allow people who have already gotten the past repeat rewards to at least get nightwave cred....
  - (0.32) Another CRED Post: How CRED Rewards really work on the inside...
  - (0.07) Maxed Skills, Tier 5 Cyberware, and Over $1M Eddies… in the Prologue. Am I Doing This Right?...
  - (0.06) The Sun Never Sets on Citadel -- Part 4...
  - (0.07) Nightwave Vol. 8 coming out...
  - (0.69) CRED be like: “Rewarding you… with what you already have!”...
  - (-0.03) How to get rid of those gambling ad banners from reward section of cred...
  - (-0.40) After just announcing the board game crowdfunding, the goal is already 1497% funded. ...


In [32]:
import json
import praw
from textblob import TextBlob
import time
from datetime import datetime, timedelta
from collections import Counter

# ----------- API KEYS AND CONFIG -----------
REDDIT_CLIENT_ID = "V8W1O2VI7wiZ1tv96LnzTg"
REDDIT_SECRET = "vlp_w9v35P-29qm-MxpyMxFB3D_mrA"
REDDIT_USER_AGENT = "PiggyLime's Market Bot"

# ----------- INIT APIS -----------
reddit = praw.Reddit(
    client_id=REDDIT_CLIENT_ID,
    client_secret=REDDIT_SECRET,
    user_agent=REDDIT_USER_AGENT
)

# ----------- SENTIMENT ANALYSIS -----------
def analyze_sentiment(text):
    """Enhanced sentiment analysis with context awareness for financial text"""
    if not text or not isinstance(text, str):
        return 0.0
    
    # Base sentiment from TextBlob
    blob = TextBlob(text)
    base_sentiment = blob.sentiment.polarity
    
    # Financial context adjustment - certain phrases have stronger sentiment in finance
    financial_boosters = {
        'crash': -0.3, 'collapse': -0.3, 'plummet': -0.3, 'tank': -0.25,
        'bearish': -0.2, 'sell-off': -0.2, 'downturn': -0.15, 'recession': -0.25,
        'bankruptcy': -0.35, 'default': -0.25, 'debt': -0.1, 'inflation': -0.1,
        
        'bullish': 0.2, 'rally': 0.2, 'surge': 0.25, 'soar': 0.25, 
        'outperform': 0.2, 'beat': 0.15, 'growth': 0.15, 'profit': 0.2,
        'buy': 0.1, 'upgrade': 0.15, 'dividend': 0.15, 'recovery': 0.15
    }
    
    # Check for booster words and adjust sentiment
    text_lower = text.lower()
    sentiment_adjustment = 0
    
    for term, value in financial_boosters.items():
        if term in text_lower:
            sentiment_adjustment += value
    
    # Adjust sentiment but keep within [-1, 1]
    adjusted_sentiment = max(min(base_sentiment + sentiment_adjustment, 1.0), -1.0)
    
    return adjusted_sentiment

# ----------- DATE CONVERSION -----------
def convert_time_filter(start_date, end_date):
    """Convert date range to appropriate Reddit time filter"""
    now = datetime.now()
    days_diff = (now - start_date).days
    
    if days_diff <= 1:
        return 'day'
    elif days_diff <= 7:
        return 'week'
    elif days_diff <= 31:
        return 'month'
    elif days_diff <= 365:
        return 'year'
    else:
        return 'all'

# ----------- REDDIT ANALYSIS -----------
def fetch_reddit(keyword, start_date=None, end_date=None, limit=15, min_score=2):
    """
    Enhanced Reddit data collection with date filtering
    
    Args:
        keyword (str): Topic to search for
        start_date (datetime): Start date for search range
        end_date (datetime): End date for search range
        limit (int): Maximum posts per subreddit
        min_score (int): Minimum upvotes required
        
    Returns:
        list: Processed Reddit posts with sentiment and metadata
    """
    print(f"\n🔍 Reddit posts on: {keyword}")
    
    # Date range formatting
    date_str = ""
    if start_date and end_date:
        date_str = f" from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}"
        print(f"📅 Time range:{date_str}")
        time_filter = convert_time_filter(start_date, datetime.now())
    else:
        time_filter = 'month'  # Default to past month
        print("📅 Time range: Past month (default)")
    
    posts = []
    
    # Finance and investment focused subreddits + general relevant ones
    subreddits = [
        # Finance & Investment - Global
        "investing", "stocks", "wallstreetbets", "StockMarket", "ValueInvesting",
        "options", "algotrading", "SecurityAnalysis", "InvestmentEducation","CreditCardsIndia",
        "dividends", "Daytrading", "StockMarket", "pennystocks", "investing_discussion",
        
        # Finance & Investment - India
        "IndiaInvestments", "IndianStreetBets", "IndiaFinance", "IndianStockMarket", "CREDclub",
        "DalalStreetTalks", "fiindia", "indianbusinessowners", "CREDclub",
        
        # Markets & Economics
        "economy", "Economics", "finance", "FinancialIndependence", "Banking",
        "GlobalMarkets", "econmonitor", "IndianEconomy", "supplychain",
        
        # Crypto & Digital Assets
        "Bitcoin", "cryptocurrency", "CryptoMarkets", "ethtrader", "CryptoCurrencyTrading",
        "CryptoTechnology", "altcoin", "defi", "NFT", "IndianCryptoMarkets",
        
        # Industry specific
        "tech", "technology", "business", "realestateinvesting", "Energy",
        "startups", "healthcare", "retailinvesting", "AutoIndustry", "renewableenergy",
        
        # General discussion
        "worldnews", "news", "india", "dataisbeautiful", "AskEconomics"
    ]
    
    # Keywords to check for relevance in the financial context
    finance_terms = ['market', 'stock', 'invest', 'price', 'share', 'trading',
                     'finance', 'dividend', 'bull', 'bear', 'crypto', 'quarter',
                     'earnings', 'portfolio', 'economy', 'fiscal', 'trend', 'growth',
                     'profit', 'loss', 'debt', 'revenue', 'valuation', 'forecast',
                     'analysis', 'sector', 'fund', 'etf', 'index', 'inflation']
    
    # Counter to track most active subreddits for this keyword
    subreddit_count = Counter()
    total_processed = 0
    post_dates = []  # To track post date distribution
    
    try:
        for sub in subreddits:
            print(f"📬 Searching r/{sub}...")
            try:
                # Use Reddit's time filter for initial filtering
                search_results = reddit.subreddit(sub).search(
                    f'"{keyword}"', 
                    sort='relevance', 
                    time_filter=time_filter, 
                    limit=limit
                )
                
                for submission in search_results:
                    total_processed += 1
                    
                    # Get submission date
                    post_date = datetime.fromtimestamp(submission.created_utc)
                    
                    # Apply date filter if specified
                    if start_date and post_date < start_date:
                        continue
                    if end_date and post_date > end_date:
                        continue
                        
                    # Track post date for analytics
                    post_dates.append(post_date)
                    
                    # Skip low quality posts
                    if submission.score < min_score:
                        continue
                    
                    # Get basic post info
                    title = submission.title
                    selftext = submission.selftext if hasattr(submission, 'selftext') else ""
                    url = submission.url
                    score = submission.score
                    created = post_date.strftime('%Y-%m-%d')
                    
                    # Get top comments for additional context
                    comments = []
                    submission.comments.replace_more(limit=0)
                    for comment in submission.comments[:3]:  # Top 3 comments
                        if comment.score > 1:  # Only include comments with upvotes
                            comments.append(comment.body)
                    
                    # Combine all text for sentiment analysis
                    full_content = f"{title}. {selftext}"
                    comment_text = " ".join(comments)
                    
                    # Check if the keyword is actually present
                    if keyword.lower() not in (full_content + comment_text).lower():
                        continue
                    
                    # Check for finance relevance (if not in primary finance subreddit)
                    primary_finance_subs = ["investing", "stocks", "wallstreetbets", "IndiaInvestments", 
                                           "cryptocurrency", "economy", "finance"]
                    if sub not in primary_finance_subs:
                        # Check if any finance term exists in the content
                        has_finance_term = any(term in (full_content + comment_text).lower() for term in finance_terms)
                        if not has_finance_term:
                            continue
                    
                    # Calculate sentiment
                    title_sentiment = analyze_sentiment(title)
                    content_sentiment = analyze_sentiment(full_content)
                    comment_sentiment = analyze_sentiment(comment_text) if comment_text else 0
                    
                    # Average sentiment with weights - title matters more
                    weighted_sentiment = (title_sentiment * 0.4 + 
                                         content_sentiment * 0.3 + 
                                         comment_sentiment * 0.3)
                    
                    # Calculate engagement score
                    engagement = submission.score + (len(comments) * 2)
                    
                    # Extract post flair if available
                    flair = submission.link_flair_text if hasattr(submission, 'link_flair_text') else None
                    
                    # Save post data
                    post_data = {
                        "title": title,
                        "content": full_content[:300] + "..." if len(full_content) > 300 else full_content,
                        "subreddit": sub,
                        "sentiment": round(weighted_sentiment, 2),
                        "score": score,
                        "comments": len(submission.comments),
                        "url": url,
                        "date": created,
                        "engagement": engagement,
                        "flair": flair
                    }
                    
                    posts.append(post_data)
                    subreddit_count[sub] += 1
                    
                # Sleep briefly to avoid hitting rate limits
                time.sleep(0.1)
                
            except Exception as e:
                print(f"Error in r/{sub}: {e}")
                continue
                
    except Exception as e:
        print(f"Error fetching Reddit posts: {e}")
    
    # Sort by engagement and sentiment (most engaging and positive/negative first)
    posts.sort(key=lambda x: (x["engagement"], abs(x["sentiment"])), reverse=True)
    
    print(f"\n✅ Processed {total_processed} Reddit posts, found {len(posts)} relevant results")
    
    if post_dates:
        earliest = min(post_dates).strftime('%Y-%m-%d')
        latest = max(post_dates).strftime('%Y-%m-%d')
        print(f"📅 Posts date range: {earliest} to {latest}")
    
    print(f"🏆 Most active subreddits: {', '.join([f'r/{sub}' for sub, count in subreddit_count.most_common(3)])}")
    
    return posts

def extract_key_phrases(posts, keyword):
    """Extract recurring themes and related terms from posts"""
    # Simple word frequency analysis to find related terms
    all_text = " ".join([post["title"] + " " + post["content"] for post in posts])
    words = all_text.lower().split()
    
    # Filter words to remove common ones and keep only relevant
    stopwords = ["the", "a", "an", "in", "on", "at", "for", "with", "by", "to", 
                "of", "is", "are", "was", "were", "and", "this", "that", "it", "from",
                "will", "would", "could", "should", "have", "has", "had", "been",
                "their", "there", "they", "them", "these", "those", "then", "than",
                "but", "not", "what", "which", "who", "when", "where", "why", "how"]
    
    filtered_words = [w for w in words if len(w) > 3 and w not in stopwords 
                     and w != keyword.lower() and w not in keyword.lower().split()]
    
    # Get top terms
    word_counts = Counter(filtered_words)
    return word_counts.most_common(15)

def extract_topics(posts):
    """Extract common topics from posts using flairs and content analysis"""
    # Get flairs
    flairs = [post["flair"] for post in posts if post.get("flair")]
    flair_counter = Counter(flairs)
    
    # Extract topics from titles using simplistic approach
    topics = []
    common_title_patterns = [
        "about", "regarding", "on", "for", "vs", "versus", "and", "-", ":"
    ]
    
    for post in posts:
        title = post["title"].lower()
        
        # Look for patterns like "X vs Y" or "About X"
        for pattern in common_title_patterns:
            if pattern in title:
                parts = title.split(pattern)
                if len(parts) >= 2:
                    for part in parts:
                        clean_part = part.strip()
                        if len(clean_part) > 3 and clean_part not in topics:
                            topics.append(clean_part)
    
    return flair_counter.most_common(5), topics[:10]

def analyze_sentiment_trends(posts):
    """Analyze sentiment trends over time"""
    # Group posts by date
    date_sentiment = {}
    for post in posts:
        date = post["date"]
        sentiment = post["sentiment"]
        
        if date not in date_sentiment:
            date_sentiment[date] = []
        
        date_sentiment[date].append(sentiment)
    
    # Calculate average sentiment per day
    daily_sentiment = {date: sum(sentiments)/len(sentiments) 
                      for date, sentiments in date_sentiment.items()}
    
    # Sort by date
    sorted_dates = sorted(daily_sentiment.keys())
    trend_data = [(date, daily_sentiment[date]) for date in sorted_dates]
    
    return trend_data

def summarize_reddit_insights(posts, keyword):
    """Generate comprehensive insights from Reddit data"""
    if not posts:
        print("\n❌ No relevant Reddit posts found")
        return
    
    # Calculate sentiment distribution
    sentiments = [post["sentiment"] for post in posts]
    positive = sum(1 for s in sentiments if s > 0.1)
    negative = sum(1 for s in sentiments if s < -0.1)
    neutral = len(sentiments) - positive - negative
    
    avg_sentiment = sum(sentiments) / len(sentiments) if sentiments else 0
    
    # Extract sentiment trend
    sentiment_trend = analyze_sentiment_trends(posts)
    trend_direction = "⬆️ IMPROVING" if len(sentiment_trend) > 1 and sentiment_trend[-1][1] > sentiment_trend[0][1] else "⬇️ DECLINING" if len(sentiment_trend) > 1 and sentiment_trend[-1][1] < sentiment_trend[0][1] else "➡️ STABLE"
    
    # Extract common themes
    key_terms = extract_key_phrases(posts, keyword)
    
    # Extract topics
    flairs, topics = extract_topics(posts)
    
    # Identify most active communities
    community_activity = Counter([post["subreddit"] for post in posts])
    top_communities = community_activity.most_common(5)
    
    # Print comprehensive summary
    print("\n" + "="*60)
    print(f"📊 MARKET INSIGHT SUMMARY: '{keyword.upper()}'")
    print("="*60)
    
    # Overall sentiment summary
    print(f"\n🌡️ SENTIMENT ANALYSIS:")
    sentiment_label = "BULLISH" if avg_sentiment > 0.2 else "BEARISH" if avg_sentiment < -0.2 else "NEUTRAL"
    print(f"- Overall market sentiment: {avg_sentiment:.2f} [{sentiment_label}]")
    print(f"- Sentiment trend: {trend_direction}")
    print(f"- Distribution: {positive} positive ({positive/len(posts)*100:.1f}%), {neutral} neutral ({neutral/len(posts)*100:.1f}%), {negative} negative ({negative/len(posts)*100:.1f}%)")
    
    # Key communities and topics
    print(f"\n🏙️ KEY COMMUNITIES:")
    for sub, count in top_communities:
        sub_posts = [p for p in posts if p["subreddit"] == sub]
        sub_sentiment = sum(p["sentiment"] for p in sub_posts) / len(sub_posts)
        print(f"- r/{sub}: {count} posts | Sentiment: {sub_sentiment:.2f}")
    
    # Main topics and themes
    print(f"\n🔑 KEY THEMES & RELATED TERMS:")
    print(f"- {', '.join([term[0] for term in key_terms[:10]])}")
    
    if flairs:
        print(f"\n🏷️ COMMON POST CATEGORIES:")
        for flair, count in flairs:
            print(f"- {flair}: {count} posts")
    
    # Overall market assessment
    print(f"\n📈 MARKET ASSESSMENT:")
    if avg_sentiment > 0.2:
        print("- The market appears POSITIVE on this topic")
        print("- Investors seem optimistic and discussions show confidence")
    elif avg_sentiment < -0.2:
        print("- The market appears NEGATIVE on this topic")
        print("- Investors express concern and discussions show caution")
    else:
        print("- The market appears MIXED/NEUTRAL on this topic")
        print("- Investors have varied opinions with no strong consensus")
    
    # Additional insights based on content analysis
    bullish_terms = ["growth", "rally", "opportunity", "undervalued", "buy", "bullish"]
    bearish_terms = ["risk", "overvalued", "crash", "sell", "bearish", "caution"]
    
    all_content = " ".join([p["title"] + " " + p["content"] for p in posts]).lower()
    
    bull_mentions = sum(all_content.count(term) for term in bullish_terms)
    bear_mentions = sum(all_content.count(term) for term in bearish_terms)
    
    if bull_mentions > bear_mentions * 1.5:
        print("- Bullish sentiment significantly outweighs bearish views")
    elif bear_mentions > bull_mentions * 1.5:
        print("- Bearish sentiment significantly outweighs bullish views")
    else:
        print("- Mixed bull/bear sentiment suggests uncertainty")
    
    # Show top posts by sentiment category
    print("\n" + "-"*60)
    print("📝 NOTABLE DISCUSSIONS")
    print("-"*60)
    
    print("\n🟢 TOP POSITIVE DISCUSSIONS:")
    positive_posts = sorted([p for p in posts if p["sentiment"] > 0.1], 
                           key=lambda x: x["sentiment"], reverse=True)[:3]
    for i, post in enumerate(positive_posts, 1):
        print(f"  {i}. r/{post['subreddit']} | {post['date']} | ⬆️ {post['score']} | 💬 {post['comments']} | 😀 {post['sentiment']}")
        print(f"     {post['title']}")
    
    print("\n🔴 TOP NEGATIVE DISCUSSIONS:")
    negative_posts = sorted([p for p in posts if p["sentiment"] < -0.1], 
                           key=lambda x: x["sentiment"])[:3]
    for i, post in enumerate(negative_posts, 1):
        print(f"  {i}. r/{post['subreddit']} | {post['date']} | ⬆️ {post['score']} | 💬 {post['comments']} | 😡 {post['sentiment']}")
        print(f"     {post['title']}")
    
    print("\n📈 MOST ENGAGING DISCUSSIONS:")
    engaging_posts = sorted(posts, key=lambda x: x["engagement"], reverse=True)[:5]
    for i, post in enumerate(engaging_posts, 1):
        print(f"  {i}. r/{post['subreddit']} | {post['date']} | ⬆️ {post['score']} | 💬 {post['comments']} | {post['sentiment']:.2f}")
        print(f"     {post['title']}")
    
    # Conclusion and recommendation
    print("\n" + "="*60)
    print("💡 CONCLUSION")
    print("="*60)
    
    if avg_sentiment > 0.3:
        print(f"The market sentiment on {keyword} is strongly POSITIVE. Community discussions indicate high confidence.")
    elif avg_sentiment > 0.1:
        print(f"The market sentiment on {keyword} is moderately POSITIVE. Discussions show cautious optimism.")
    elif avg_sentiment < -0.3:
        print(f"The market sentiment on {keyword} is strongly NEGATIVE. Community discussions express significant concern.")
    elif avg_sentiment < -0.1:
        print(f"The market sentiment on {keyword} is moderately NEGATIVE. Discussions indicate caution.")
    else:
        print(f"The market sentiment on {keyword} is NEUTRAL or MIXED. No clear consensus has emerged.")

def parse_date(date_str):
    """Parse date string into datetime object"""
    try:
        # Try various formats
        formats = [
            '%Y-%m-%d',
            '%d/%m/%Y',
            '%m/%d/%Y',
            '%d-%m-%Y',
            '%m-%d-%Y',
            '%d %b %Y',
            '%b %d %Y'
        ]
        
        for fmt in formats:
            try:
                return datetime.strptime(date_str, fmt)
            except ValueError:
                continue
        
        # If no format works, raise exception
        raise ValueError(f"Couldn't parse date: {date_str}")
    except Exception as e:
        print(f"Error parsing date: {e}")
        return None

# ----------- MAIN WORKFLOW -----------
def get_market_insights(query, start_date=None, end_date=None):
    """Get market insights for a query within a date range"""
    print(f"\n🔎 Researching market insights for: '{query}'")
    
    # Parse dates if provided
    start_date_obj = parse_date(start_date) if start_date else None
    end_date_obj = parse_date(end_date) if end_date else None
    
    # Fetch and analyze Reddit posts
    reddit_posts = fetch_reddit(query, start_date_obj, end_date_obj)
    
    # Summarize Reddit insights
    summarize_reddit_insights(reddit_posts, query)
    
    return reddit_posts

# ----------- Run Example -----------
if __name__ == '__main__':
    print("🚀 MARKET SENTIMENT ANALYZER 🚀")
    print("-----------------------------------")
    user_query = input("Enter market topic to research: ")
    
    # Optional date filtering
    use_dates = input("Filter by date range? (y/n): ").lower() == 'y'
    
    if use_dates:
        start_date = input("Enter start date (YYYY-MM-DD or leave blank for none): ")
        end_date = input("Enter end date (YYYY-MM-DD or leave blank for none): ")
        
        # Handle empty inputs
        start_date = start_date if start_date.strip() else None
        end_date = end_date if end_date.strip() else None
        
        get_market_insights(user_query, start_date, end_date)
    else:
        get_market_insights(user_query)

🚀 MARKET SENTIMENT ANALYZER 🚀
-----------------------------------


Enter market topic to research:  ipl
Filter by date range? (y/n):  n



🔎 Researching market insights for: 'ipl'

🔍 Reddit posts on: ipl
📅 Time range: Past month (default)
📬 Searching r/investing...
📬 Searching r/stocks...
📬 Searching r/wallstreetbets...
📬 Searching r/StockMarket...
📬 Searching r/ValueInvesting...
📬 Searching r/options...
📬 Searching r/algotrading...
📬 Searching r/SecurityAnalysis...
📬 Searching r/InvestmentEducation...
📬 Searching r/CreditCardsIndia...
📬 Searching r/dividends...
📬 Searching r/Daytrading...
📬 Searching r/StockMarket...
📬 Searching r/pennystocks...
📬 Searching r/investing_discussion...
📬 Searching r/IndiaInvestments...
📬 Searching r/IndianStreetBets...
📬 Searching r/IndiaFinance...
📬 Searching r/IndianStockMarket...
📬 Searching r/CREDclub...
📬 Searching r/DalalStreetTalks...
📬 Searching r/fiindia...
Error in r/fiindia: Redirect to /subreddits/search
📬 Searching r/indianbusinessowners...
Error in r/indianbusinessowners: Redirect to /subreddits/search
📬 Searching r/CREDclub...
📬 Searching r/economy...
📬 Searching r/Economics